In [6]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import  train_test_split
from sklearn.ensemble        import  RandomForestClassifier
from sklearn.metrics         import  roc_auc_score
import gc


In [2]:
path = "mnt/ssd/kaggle-talkingdata2/competition_files/"

dtypes = {'ip'           : 'uint32',
          'app'          : 'uint16',
          'device'       : 'uint16',
          'os'           : 'uint16',
          'channel'      : 'uint16',
          'is_attributed': 'uint8' ,
          'click_id'     : 'uint32'
    
         }
print('loading train data...')

train_df = pd.read_csv(path+"train.csv", dtype = dtypes, usecols = ['ip','app','device','os','channel','click_time','is_attributed'])
# total observation : 184,903,891

loading train data...


In [4]:
train_df['hour'] = pd.to_datetime(train_df.click_time).dt.hour.astype('uint8')
train_df['day']  = pd.to_datetime(train_df.click_time).dt.day.astype('uint8')
train_df['min']  = pd.to_datetime(train_df.click_time).dt.minute.astype('uint8')
train_df.drop(['click_time'], axis=1, inplace=True)
y = train_df['is_attributed']

train_df.drop(['is_attributed'], axis=1, inplace=True)

In [7]:
x_train, x_test, y_train, y_test =train_test_split(train_df, y, test_size=0.2, random_state =99)
x_train, x_cv, y_train, y_cv =train_test_split(x_train, y_train, test_size=0.2, random_state =99)
gc.collect()

29

In [ ]:
# random forest with parameter tuning

alpha = [100,200,400,700,1000]
max_depth = [5, 10]
cv_roc_score_array = []
for i in alpha:
    for j in max_depth:
        print("for n_estimators =", i,"and max depth = ", j)
        clf = RandomForestClassifier(n_estimators=i, criterion='gini', max_depth=j, random_state=42, n_jobs=-1,class_weight ="balanced")
        clf.fit(x_train, y_train)
        y_predicted = clf.predict(x_cv)
        score = roc_auc_score(y_cv, y_predicted)
        cv_roc_score_array.append(score)
        print("Roc Score :",score) 


best_alpha = np.argmax(cv_roc_score_array)
clf = RandomForestClassifier(n_estimators=alpha[int(best_alpha)], criterion='gini', max_depth=max_depth[int(best_alpha)], random_state=42, n_jobs=-1,class_weight ="balanced")
clf.fit(x_train, train_y)

y_predict = clf.predict(x_train)
print('For values of best alpha = ', alpha[best_alpha], "The roc score is:",metrics.roc_auc_score(y_train, y_predict))
y_predict = clf.predict(x_cv)
print('For values of best alpha = ', alpha[best_alpha], "The roc score is:",metrics.roc_auc_score(y_cv, y_predict))
y_predict = clf.predict(x_test)
print('For values of best alpha = ', alpha[best_alpha], "The roc score is:",metrics.roc_auc_score(y_test, y_predict))

for n_estimators = 100 and max depth =  5
